# Value Investing Indicators from SEC Filings

### Data Source: https://www.sec.gov/dera/data/financial-statement-data-sets.html



In [1]:
import pandas as pd
import os
import shutil
import glob
import sys
import warnings
import functools
from functools import reduce
import os 

pd.set_option('display.max_columns', 999)

warnings.simplefilter("ignore")

os.chdir("..")

dir_root = os.getcwd()


In [2]:
dir_raw = dir_root + u"/sec_filings/Raw"

cik_lookup = pd.read_csv(dir_root + u"/sec_filings/cik_ticker.csv", usecols=['CIK', 'Ticker'], sep="|")


In [3]:
# num_tags = ["PreferredStockValue", "AssetsCurrent", "Liabilities", "EarningsPerShareBasic", "CommonStockSharesOutstanding", "LiabilitiesCurrent", "EarningsPerShareBasic", "SharePrice", "StockholdersEquity", "PreferredStockValue", "CommonStockSharesOutstanding", "NetIncomeLoss", "GrossProfit", "SalesRevenueNet","StockRepurchasedAndRetiredDuringPeriodShares"]

files_num = sorted(glob.glob(dir_raw + u'/*_num.txt'))

files_sub = sorted(glob.glob(dir_raw + u'/*_sub.txt'))

files_pre = sorted(glob.glob(dir_raw + u'/*_pre.txt'))

sec_df = pd.DataFrame([])

for num, sub, pre in zip(files_num, files_sub, files_pre):
    df_num = pd.read_csv(num, sep='\t', dtype=str, encoding = "ISO-8859-1")
    df_sub = pd.read_csv(sub, sep='\t', dtype=str, encoding = "ISO-8859-1")
    df_pre = pd.read_csv(pre, sep='\t', dtype=str, encoding = "ISO-8859-1")
    # df_num = df_num[df_num['tag'].isin(num_tags)]
    df_pre = df_pre.merge(df_num, on=['adsh', 'tag', 'version'], sort=True)
    sec_merge = df_sub.merge(df_pre, on='adsh', how="inner", sort=True)
    sec_merge = sec_merge[sec_merge['form'] == "10-Q"]
    sec_merge = sec_merge[sec_merge['stmt'] == "BS"]
    sec_df = sec_df.append(sec_merge)


In [4]:
sec_curated = sec_df.sort_values(by='ddate').drop_duplicates(subset=['adsh', 'tag', 'version'], keep='last')

sec_curated = sec_curated[['adsh', 'ddate','version','filed', 'form', 'fp', 'fy', 'fye', 'instance', 'period', 'tag', 'uom', 'value']]

sec_curated.to_csv(dir_root + r'/test.csv', index=False)

In [30]:
sec_curated = sec_curated.drop_duplicates(subset=['instance'])
sec_curated[sec_curated['tag'] == 'Share'].dropna()


,adsh,ddate,version,filed,form,fp,fy,fye,instance,period,tag,uom,value


In [6]:
drop_cols = ['adsh', 'ddate','version','filed', 'form', 'fp', 'fy', 'fye', 'period', 'tag', 'uom']

PreferredStockValue = sec_curated[sec_curated["tag"] == "PreferredStockValue"].rename(columns={"value": "PreferredStockValue"}).drop(columns=drop_cols)

AssetsCurrent = sec_curated[sec_curated["tag"] == "AssetsCurrent"].rename(columns={'value': 'AssetsCurrent'}).drop(columns=drop_cols)

Liabilities = sec_curated[sec_curated["tag"] == "Liabilities"].rename(columns={'value': 'Liabilities'}).drop(columns=drop_cols)

EarningsPerShareBasic = sec_curated[sec_curated["tag"] == "EarningsPerShareBasic"].rename(columns={'value': 'EarningsPerShareBasic'}).drop(columns=drop_cols)

CommonStockSharesOutstanding = sec_curated[sec_curated["tag"] == "CommonStockSharesOutstanding"].rename(columns={'value': 'CommonStockSharesOutstanding'}).drop(columns=drop_cols)

LiabilitiesCurrent = sec_curated[sec_curated["tag"] == "LiabilitiesCurrent"].rename(columns={'value': 'LiabilitiesCurrent'}).drop(columns=drop_cols)

EarningsPerShareBasic = sec_curated[sec_curated["tag"] == "EarningsPerShareBasic"].rename(columns={'value': 'EarningsPerShareBasic'}).drop(columns=drop_cols)

SharePrice = sec_curated[sec_curated["tag"] == "SharePrice"].rename(columns={'value': 'SharePrice'}).drop(columns=drop_cols)

StockholdersEquity = sec_curated[sec_curated["tag"] == "StockholdersEquity"].rename(columns={'value': 'StockholdersEquity'}).drop(columns=drop_cols)

PreferredStockValue = sec_curated[sec_curated["tag"] == "PreferredStockValue"].rename(columns={'value': 'PreferredStockValue'}).drop(columns=drop_cols)

CommonStockSharesOutstanding = sec_curated[sec_curated["tag"] == "CommonStockSharesOutstanding"].rename(columns={'value': 'CommonStockSharesOutstanding'}).drop(columns=drop_cols)

NetIncomeLoss = sec_curated[sec_curated["tag"] == "NetIncomeLoss"].rename(columns={'value': 'NetIncomeLoss'}).drop(columns=drop_cols)

GrossProfit = sec_curated[sec_curated["tag"] == "GrossProfit"].rename(columns={'value': 'GrossProfit'}).drop(columns=drop_cols)

SalesRevenueNet = sec_curated[sec_curated["tag"] == "SalesRevenueNet"].rename(columns={'value': 'SalesRevenueNet'}).drop(columns=drop_cols)

StockRepurchased = sec_curated[sec_curated["tag"] == "StockRepurchasedAndRetiredDuringPeriodShares"].rename(columns={'value': 'StockRepurchased'}).drop(columns=drop_cols)


In [15]:
cols = ['adsh', 'ddate','version','filed', 'form', 'fp', 'fy', 'fye', 'instance', 'period', 'tag', 'uom']

sec_final = sec_curated[cols]

dfs = [sec_curated, PreferredStockValue, AssetsCurrent, Liabilities, EarningsPerShareBasic, 
CommonStockSharesOutstanding, LiabilitiesCurrent, EarningsPerShareBasic, 
SharePrice, StockholdersEquity, PreferredStockValue, CommonStockSharesOutstanding, 
NetIncomeLoss, GrossProfit, SalesRevenueNet,StockRepurchased]

df_final = reduce(lambda left,right: pd.merge(sec_curated,right,on='instance'), dfs)

In [28]:
dfs_final = [df.set_index('instance') for df in dfs]
x = pd.concat(dfs_final, axis=1)

x['SharePriceBasic'].dropna()


KeyError: 'SharePriceBasic'

# Benjamin Graham 

## Formulas:

* NCAVPS = CurrentAssets - (Total Liabilities + Preferred Stock) ÷ Shares Outstanding
    * Less than 1.10 

* Debt to Assets = Current Assets / Current Liabilities
    * Greater than 1.50

* Price / Earnings per Share ratio 
    * Less than 9.0

* PRICE TO BOOK VALUE = (P/BV) 
    * Where BV = (Total Shareholder Equity−Preferred Stock)/ Total Outstanding Shares
    * Less than 1.20. P/E ratios

## References:

Benjamin Graham rules: https://cabotwealth.com/daily/value-investing/benjamin-grahams-value-stock-criteria/ 

Benjamin Graham rules Modified: https://www.netnethunter.com/16-benjamin-graham-rules/ 



In [ ]:

sec_df['NCAVPS'] = sec_df['AssetsCurrent'] - (sec_df['Liabilities'] + sec_df[
'PreferredStockValue']) / sec_df['CommonStockSharesOutstanding']

sec_df['DebtToAssets'] = sec_df['AssetsCurrent'] / sec_df['LiabilitiesCurrent']

sec_df['PE'] = sec_df['SharePrice'] / sec_df['EarningsPerShareBasic'] 

sec_df['PBV'] = sec_df['SharePrice'] / ((sec_df['StockholdersEquity'] - sec_df['PreferredStockValue']) / sec_df['CommonStockSharesOutstanding'])


# Warren Buffet Rules 

## Formulas

* Debt/Equity= Total Liabilities / Total Shareholders’ Equity 
    * Less than 1 and ROE is greater than 10%

* Return on Earnings = (Net Income / Stock Holders Equity)
    * Is Positive

* Gross Profit Margin = Gross Profit / Revenue 
    * Greater than 40% 
​
* Quarter over Quarter EPS 
    * Greater than 10

* Stock Buybacks
    * Greater than last period

## References: 
https://www.oldschoolvalue.com/tutorial/this-is-how-buffett-interprets-financial-statements/

In [ ]:

sec_df['DebtEquity'] = sec_df['Liabilities'] / sec_df['StockholdersEquity']

sec_df['ReturnEarnings'] = sec_df['NetIncomeLoss'] / sec_df['StockholdersEquity']

sec_df["GrossProfitMargin"] = sec_df['GrossProfit'] / sec_df['SalesRevenueNet']

sec_df["EPS"] = sec_df["EarningsPerShareBasic"]

sec_df["StockBuybacks"] = sec_df["StockRepurchasedAndRetiredDuringPeriodShares"]